In [1]:
import pandas as pd
import os
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from difflib import SequenceMatcher
import operator
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [3]:
#Pull ingredient list + classification
df = pd.read_csv('recipe_app/data/recipe_ingredients_new.csv',index_col=False, encoding='latin1')
df_new = pd.read_csv('recipe_app/data/final_recipe_list_categorized.csv',index_col=False, encoding='utf-8')

## Test area with three recipes

In [ ]:
# Need list of metrics for standardizing

#you buy oz or mL
#pounds or grams

# metrics = ['tablespoon', 'teaspoon', 'tbsp', 'tsp', 'cup', 'ounce', 'oz', \
#            'quart', 'qt', 'pt', 'pint', 'gallon', 'gal', 'pount', 'lb', 'g', \
#            'gram', 'kilogram', 'kg', 'liter', 'L', 'millileter', 'mL']

In [ ]:
# recipe_ingredients = ['ounce country-style bread, crusts removed', 'tablespoons olive oil, divided', \
#                       'scallions', 'salt, freshly ground pepper', 'tablespoons fresh lemon juice', \
#                       'tablespoons unseasoned rice vinegar', 'tablespoons Dijon mustard', \
#                       'tablespoons mayonnaise, preferably Hellmann’s', \
#                       'small rotisserie chicken, meat pulled from bone', 'radishes, trimmed, cut into wedges', \
#                       'head of Bibb lettuce, leaves separated', 'avocado, sliced, divided', 'pound ground lamb', \
#                       'cup yellow or red onion, ﬁnely chopped', 'cloves garlic, minced', 'teaspoons fresh ginger, ﬁnely grated', \
#                       'teaspoon garam masala', 'teaspoon curry powder', 'teaspoon paprika', 'Salt', 'Pepper', 'ramps, just the greens, minced', 
#                       'tablespoons extra-virgin olive oil', 'teaspoons lemon juice', 'cup yogurt', 'large ﬂour tortillas or wraps', \
#                       'Persian cucumbers, sliced very thin on a mandoline', 'red onion, thinly sliced', \
#                       'Fresh cilantro sprigs, about 1/4 bunch', 'Greens or microgreens', 'Nonstick vegetable oil spray', \
#                       'cups all-purpose flour', 'cup sugar', 'teaspoons baking powder', 'teaspoon kosher salt', \
#                       'large eggs', 'cups ricotta', 'teaspoon vanilla extract', 'cup (1 stick) unsalted butter, melted', \
#                       'cup frozen raspberries or blackberries, divided', 'ounces linguine', 'Kosher salt', \
#                       'tablespoons unsalted butter, divided', 'tablespoons olive oil, divided', 'cup minced shallots', \
#                       'teaspoon minced garlic', 'Fresno chiles, red jalapeños, or red Thai chiles, seeded, sliced into thin rounds', \
#                       'tablespoons (or more) fresh lemon juice, divided', 'teaspoons (packed) finely grated lemon zest, divided', \
#                       'Freshly ground black pepper', 'ounces cooked, shelled Dungeness crab, king crab, or jumbo lump crabmeat, picked over for shells', \
#                       'cup fresh mint leaves, gently torn, divided']

In [ ]:
#Find words in metrics list for recipe list generation

def strip_measurements(x, metrics):
    found_word = ''
    for word in x.split(' '):
        if word in metrics:
            found_word = word
    return found_word

new_df['metric'] = new_df['recipe_ingredients'].apply(lambda x: strip_measurements(x, metrics))

In [ ]:
#Find longest matching string in 'ingredient' field in database

def standardize_ingredients(row, ingredients):
    possible_standard = []
    for ingredient in ingredients:
        if ingredient in row['recipe_ingredients']:
            possible_standard.append(ingredient)
    if len(possible_standard) == 0:
        return row['recipe_ingredients']
    else:
        return max(possible_standard, key=len)

new_df['recipe_ingredients'] = new_df.apply(lambda row: standardize_ingredients(row, df['ingredients']), axis=1)

In [ ]:
#create list of ngrms turned into individual strings
#['I', 'am', 'I am']

def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
import re
def strip_word(row):
    try:
        new_recipe_ingredients = row['recipe_ingredients'].replace(row['metric'], '')
        new_recipe_ingredients = re.sub('[^A-Za-z0-9]+', ' ', new_recipe_ingredients)
    except:
        new_recipe_ingredients = re.sub('[^A-Za-z0-9]+', ' ', row['recipe_ingredients'])
    return new_recipe_ingredients

new_df['recipe_ingredients'] = new_df.apply(lambda row: strip_word(row), axis=1)

In [ ]:
#convert to ounces

def gallon_to_quart(n_gallon):
    n_quarts = n_gallon * 4
    return n_quarts

def quart_to_pints(n_quarts):
    n_pints = n_quarts * 2
    return n_pints
    
def pint_to_cups(n_pints):
    n_cups = n_pints * 2
    return n_cups

def cup_to_ounces(n_cups):
    n_ounces = n_cups * 8
    return n_ounces

def tsps_to_tbs(n_tsps):
    n_tbs = float(n_tsps)/3
    return n_tbs

def tbs_to_cups(n_tbs):
    n_cups = float(n_tbs)/16
    

#convert to mL

def ounces_to_mL(n_ounces);
    n_mLs = n_ounces * 29.5735
    return n_mLs

#convert pounds to kilos
def pounds_to_kilos(n_pounds):
    n_kilos = n_pounds / 2.205
    return n_kilos

#convert kilos to pounds
def kilos_to_pounds(n_kilos):
    n_pounds = n_kilos * 2.205
    return n_pounds

#convert ounces to grams
def ounces_to_grams(n_ounces):
    n_grams = n_ounces * 28.35
    return n_grams


#convert pounds to ounces
def pounds_to_ounces(n_pounds):
    n_ounces = n_pounds * 16
    return n_ounces


In [ ]:
#            'quart', 'qt', 'pt', 'pint', 'gallon', 'gal', 'pount', 'lb', 'g', \
#            'gram', 'kilogram', 'kg', 'liter', 'L', 'millileter', 'mL']
def metric_aggregate(metric, quantity, desired_metric):
    desired_quantity = metric
    if metric in ['tablespoon', 'tbsp']:
        if desired_metric == 'oz':
            desired_quantity = cup_to_ounces(tbs_to_cups(quantity))
        else:
            desired_quantity = ounces_to_mL(cup_to_ounces(tbs_to_cups(quantity)))
    if metric in ['teaspoon', 'tsp']:
        if desired_metric == 'oz':
            if desired_metric == 'oz':
            desired_quantity = cup_to_ounces(tbs_to_cups(tsps_to_tbs(quantity)))
        else:
            desired_quantity = ounces_to_mL(cup_to_ounces(tbs_to_cups(tsps_to_tbs(quantity))))
    if metric in ['ounce', 'oz']:
        if desired_metric == 'oz':
            desired_quantity = quantity
        else:
            desired_quantity = ounces_to_mL(quantity)
    if metric in ['quart', 'qt']:
        if desired_metric == 'oz':
            desired_quantity = cups_to_ounces(pints_to_cups(quart_to_pints(quantity)))
        else:
            desired_quantity = ounces_to_mL(cups_to_ounces(pints_to_cups(quart_to_pints(quantity))))
            
        
 


## Categorize ingredients

In [107]:
#Check ingredient and string match with already classified ingredients to find close match (to then classify)

def seq_matcher(string_one, df):
    x = string_one
    for index, value in df.iterrows():
        s = SequenceMatcher(None, string_one, value['ingredient'])
        if s.ratio() > 0.75:
            x = value['ingredient']
        else:
            pass
    print string_one, x
    return x

In [7]:
#Taking the columns I need

df = df[['ingredients', 'category']]
df_new = df_new[['ingredient', 'category']]

In [30]:
#unicode to ascii

import re
def reg(x):
    try:
        x = " ".join(re.findall("[a-zA-Z]+", x))
    except:
        x = x
    return x
df_new['ingredient'] = df_new['ingredient'].apply(lambda x: reg(x))
df_new['ingredient'] = [str(line).decode('utf-8').strip() for line in df_new['ingredient']]

In [15]:
#Lemmatize ingredients strings and lowercase to merge

df['ingredients'] = df['ingredients'].apply(lambda x: ', '.join([lmtzr.lemmatize(y).lower() for y in x.split(' ')]).replace(',', ''))
df_new['ingredient'] = df_new['ingredient'].apply(lambda x: ', '.join([lmtzr.lemmatize(y).lower() for y in str(x).split(' ')]).replace(',', ''))

In [34]:
#Merge on ingredients and drop duplicates

merged_df = df_new.merge(df, left_on='ingredient', right_on='ingredients', how='outer').drop_duplicates()

In [35]:
#Check which ingredients were merged and seperate still not classified ingredients

not_classified = merged_df[merged_df['category_x'] != merged_df['category_x']]
alreay_classified = merged_df[merged_df['category_x'] == merged_df['category_x']]

In [ ]:
not_classified['newest_category'] = not_classified['ingredients'].apply(lambda x: seq_matcher(x, df_new[['ingredient']]))

onion monin
olive oil olive oil
water water
garlic gari
garlic clove garlic chive
ground black pepper ground red pepper
all-purpose flour all purpose flour
large egg large egg
ground cumin ground chicken
extra-virgin olive oil extra-virgin olive oil
oil oil
red bell pepper red pepper
purple onion purple corn
grated parmesan cheese parmesan cheese
jalapeno chilies jalapeno chilies
baking powder baking paper
dried oregano dried bean
chopped cilantro fresh chopped cilantro fresh
fresh lemon juice lemon juice
diced tomato dried tomato
minced garlic minced garlic
fresh parsley french parsley
chicken broth chicken broth
fresh lime juice persian lime juice
chopped onion chopped onion
ground cinnamon ground almond
dry white wine white wine
green bell pepper green pepper
cilantro leaf cilantro leaf
cilantro cilantro root
fresh basil fresh milk
boneless skinless chicken breast boneless skinless chicken breast
boneless skinless chicken breast boneless skinless chicken breast
flat leaf parsley cur

poblano chile bola chile
polenta polenta
cardamom pod cardamom
smoked sausage smoked sable
peppercorn pepperoni
grated orange bigarade orange
mascarpone macaroon
gochujang base gochujang base
seasoning taco seasoning
fresh tomato english tomato
cooked white rice enriched white rice
pecan pemmican
parsley sprig parsley sprig
seasoning salt canning salt
fresh parsley leaf fresh parsley leaf
ground cayenne pepper ground red pepper
artichoke heart artichoke heart
fresh tarragon fresh tarragon
chopped walnut chopped walnut
fat free milk fat free milk
tequila tequila
lime zest lime zest
panko breadcrumb panko breadcrumb
non-fat sour cream tofu sour cream
chicken wing chicken egg
salsa verde salsa verde
white bread ship bread
chili pepper wild pepper
italian parsley leaf italian parsley
garlic chili sauce chili sauce
dry yeast dry yeast
dried apricot candied apricot
whole peeled tomato whole peeled tomato
dashi nashi
ginger paste ginger paste
mint sprig mint sprig
dried shiitake mushroom shit

italian turkey sausage italian sausage
chutney chunky
broccoli rabe broccoli romanesco
anaheim chile anaheim chile
edamame edamame
cheese tortellini cheese tortellini
cod fillet cod fillet
reduced fat sharp cheddar cheese reduced fat sharp cheddar cheese
sage usuage
turbinado turbinado
lobster lobster roe
shredded parmesan cheese parmesan cheese
extra sharp cheddar cheese extra sharp cheddar cheese
fusilli fusilli
dried cranberry dried cranberry
capsicum capsicum
diced red onion dried minced onion
fruit jakfruit
roast red pepper drain roast red pepper drain
chicken bouillon granule chicken bouillon granule
beef stew meat veal stew meat
fresh herb fresh chevre
vinaigrette vinaigrette
thai green curry paste red curry paste
fresh sage fresh pasta
chickpea flour chickpea flour
plain whole-milk yogurt plain whole-milk yogurt
yellow pepper yellow pea
bread slice breadstick
pitted date pitted date
rubbed sage rubbed sage
celery salt celery salt
wild mushroom woodear mushroom
dried tarragon le

jam jam
fajita seasoning mix fajita seasoning mix
ricotta salata ricotta salata
crab meat crabeater
sirloin sirloin
raw sugar bar sugar
japanese soy sauce japanesesoy sauce
toasted pecan toasted pecan
part-skim mozzarella part-skim mozzarella
pasta shell taco shell
cooked quinoa cooked quinoa
pork shoulder roast pre carved shoulder roast
candy candy
guinness beer guinness
shucked oyster shucked oyster
veggie veggie
pie dough pie dough
reduced fat monterey jack cheese reduced fat monterey jack cheese
lasagna noodle cooked and drained lasagna noodle cooked and drained
hot chili powder hot chili oil
whole wheat spaghetti whole wheat pasta
lean ground turkey ground turkey
medium shrimp uncook medium shrimp uncook
mexican beer mexican tea
cooky cooky
tilapia tilapia
sesame steamer
olive oil cooking spray olive oil spray
full fat coconut milk full fat coconut milk
kielbasa kielbasa
spaghettini spaghetti
pure maple syrup maple syrup
whole wheat pastry flour whole wheat pasta
apricot half apri

duck drumstick drumstick
beef steak tube steak
frozen mixed vegetable frozen mixed vegetable
no-salt-added black bean salted black bean
frozen edamame bean frozen edamame bean
small curd cottage cheese small curd cottage cheese
nori sheet nori sheet
tart apple spartan apple
sea salt flake spelt flake
reduced fat coconut milk reduced fat coconut milk
lean beef leaf beet
baby back rib pork baby back rib
brewed espresso brewed espresso
maggi maggi
barbecued pork barbecue rib
dill tip dill tip
meatball matzo ball
medium egg noodle medium egg noodle
chopped ham pumped ham
gin gin
kirsch kirsch
red capsicum red capsicum
chanterelle winter chanterelle
toasted slivered almond slivered almond
old-fashioned oat old fashioned oatmeal
shredded mild cheddar cheese shredded mild cheddar cheese
whole wheat breadcrumb whole wheat berry
napa cabbage leaf nappa cabbage
quick-cooking oat quick cooking tapioca
lager laver
fire roasted diced tomato fire roasted diced tomato
black tea black pasta
dried mush

...

kipper kipper
seasoning rub seasoning rub
toasted nut toasted nut
mesquite flavored seasoning mix mesquite flavored seasoning mix
fat free instant chocolate pudding mix fat free instant chocolate pudding mix
shredded low-fat sharp cheddar shredded low-fat sharp cheddar
butter cake mix butter cake
cinnamon graham cracker graham cracker
lemon twist lemon twist
domino confectioner sugar confectioner s sugar
red wine vinaigrette red wine vinegar
holland house white cooking wine holland house white cooking wine
diet orange soda diet orange soda
wish bone guacamol ranch dress wish bone guacamol ranch dress
heinz worcestershire sauce worcestershire sauce
low sodium stock low sodium stock
whole wheat cereal whole wheat berry
buttermilk ranch dressing buttermilk ranch dressing
large sausage casing sausage casing
white almond bark white almond bark
meat extract stevia extract
challenge butter challenge butter
alexia waffle fry alexia waffle fry
boneless beef top round steak boneless beef top rou

yam noodle soy noodle
kraft reduced fat shredded mozzarella cheese kraft reduced fat shredded mozzarella cheese
taco bellã£âã¢â® thick & chunky medium salsa taco bellã£âã¢â® thick & chunky medium salsa
rub seasoning rub seasoning
castelvetrano olive castelvetrano olive
tony chachere's seasoning tony chachere's seasoning
beef shin beef shank
candied jalapeno thai jalapeno
tomato basil feta tomato basil feta
french's spicy brown mustard french's spicy brown mustard
chamomile tea chamomile
ataulfo ataulfo
whole grain rotini whole grain rice
knox gelatin powder knox gelatin powder
do chua do chua
camellia red kidney bean red kidney bean
fruitcake fruitcake
amaranth seed amaranth
vanilla cake mix vanilla cake mix
dried pineapple candied pineapple
boneless steak boneless ham
duck bone duck bone
low-fat whole wheat tortilla low-fat whole wheat tortilla
potato puree tomato puree
long green chili pepper long green pepper
vegeta seasoning taco seasoning
tupelo honey tupelo honey
fruit juice 

filetta filet steak
fin-de-siecle fin-de-siecle
finlandia swiss finlandia swiss
finn finn
fiore sardo fiore sardo cheese
fleur du maquis fleur du maquis
flor de guia flor de guia
flower marie flower marie
folded folded
folded cheese with mint folded cheese with mint
fondant de brebis fondant de brebis
fontainebleau fontainebleau
fontal fontal
fontina val d'aosta fontina val d'aosta
formaggio di capra formaggio di capra
fougerus fougerus
four herb gouda four herb gouda
fourme d' ambert fourme d' ambert
fourme de haute loire fourme de haute loire
fourme de montbrison fourme de montbrison
fresh jack fresh bacon
fresh ricotta fresh ricotta
fresh truffle fresh truffle
fribourgeois fribourgeois
friesekaas friesekaas
friesian friesian
friesla friesla
frinault frinault
fromage a raclette fromage a raclette
fromage corse fromage corse
fromage de montagne de savoie fromage de montagne de savoie
fromage frais fromage frais
fruit cream cheese cream cheese
frying cheese grating cheese
fynbo fynbo
g

texas goat cheese fresh goat cheese
tibet tibet
tillamook cheddar tillamook cheddar
timboon brie timboon brie
toma tomato
tomme brulee tomme brulee
tomme d'abondance tomme d'abondance
tomme de chevre bleu de chevre
tomme de roman tomme de roman
tomme de savoie tomme de savoie
tomme de chouans tomme de chouans
tommes tommes
torta del casar torta del casar
toscanello toscanelle olive
touree de l'aubier touree de l'aubier
tourmalet tourmalet
trappe (veritable) trappe (veritable)
trois cornes de vendee trois cornes de vendee
tronchon tronchon
trou du cru trou du cru
truffe truffles
tupi tupi
turunmaa turunmaa
tymsboro tymsboro
tyn grug tyn grug
tyning tyning
ubriaco ubriaco
ulloa ulloa
vacherin-fribourgeois vacherin-fribourgeois
valencay valencay
vasterbottenost vasterbottenost
venaco venaco
vendomois vendomois
vermont cheddar vermont cheddar
vieux corse vieux corse
vignotte vignotte
vulscombe vulscombe
waimata farmhouse blue waimata farmhouse blue
waterloo watermelon
weichkaese weichkaese

collective farm woman melon collective farm woman melon
european cantaloupe european cantaloupe
north american cantaloupe north american cantaloupe
sprite melon sprite melon
tigger melon tigger melon
abiu abiu
african custard-apple mexican custard apple
african locust bean african locust bean
american persimmon american persimmon
annona conica annona conica
babaco babaco
bacuri bacardi
bael bael
bilimbi bilimbi
biribã¢â¬â¡ biribã¢â¬â¡
black apple black salt
brazilian guava brazilian guava
burahol burahol
burdekin plum burdekin plum
burmese grape burmese grape
button mangosteen button mangosteen
cacao cacao
cainito caimito
canistel anise
cawesh cawesh
cempedak cempedak
charichuelo charichuelo
cluster fig cluster fig
costa rican guava costa rican guava
courbaril courbaril
cupuaã¢âu cupuaã¢âu
davidson's plum davidson's plum
dead man's finger dead man's finger
desert fig desert fig
doub palm fruit doub palm fruit
duguetia confinis duguetia confinis
duguetia spixiana duguetia spixiana
e

## Scraping data from Food Dictionary

In [ ]:
# #Scraper

# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import urllib2
# import unicodedata

# class food_dictionary:
#     def __init__(self):
#         self.df = pd.DataFrame(columns = ['ingredient', 'category'])

#     def turn_to_df(self, category, url):
#         quote_page = url
#         page = requests.get(quote_page)
#         soup = BeautifulSoup(unicodedata.normalize('NFKD', page.text), 'html.parser')
#         x = soup.find_all("b")
#         recipe_list = []
#         for y in x:
#             recipe_list.append(y.text)
#         recipe_list = self.clean_list(recipe_list)
#         df = pd.DataFrame(recipe_list, columns=['ingredient'])
#         df['category'] = category
#         self.df = self.df.append(df)
        
#     def clean_list(self, recipe_list):
#         new_list = []
#         for x in recipe_list:
#             x = x.split('Notes:', 1)[0]
#             x = x.split('See ', 1)[0]
#             if ':' in x:
#                 pass
#             else:
#                 if '=' in x:
#                     for equivalent in x.split('='):
#                         if equivalent == u'':
#                             pass
#                         else:
#                             new_list.append(equivalent.replace('\n', '').replace(u'\xa0', u'').strip().replace('  ', ' '))
#                 else:
#                     if x == u'':
#                         pass
#                     else:
#                         new_list.append(x.replace('\n', ' ').replace(u'\xa0', u'').strip().replace('  ', ' '))
#         return new_list[1:]

In [ ]:
# #List of urls and classification of ingredients on page

# scrape_list = [('vegetable-tuber', 'http://www.foodsubs.com/Tubers.html'), \
#                ('vegetable-root', 'http://www.foodsubs.com/Roots.html'), \
#                ('vegetable-potatoes','http://www.foodsubs.com/Potatoes.html'), \
#                ('vegetable-sweet potatoes','http://www.foodsubs.com/Sweetpotatoes.html'), \
#                ('vegetable-stalk', 'http://www.foodsubs.com/Stalk.html'), \
#                ('vegetable-onions', 'http://www.foodsubs.com/Onionsgreen.html'), \
#                ('vegetable-onions', 'http://www.foodsubs.com/Onionsdry.html'), \
#                ('vegetable-garlic', 'http://www.foodsubs.com/Garlic.html'), \
#                ('vegetable-ginger', 'http://www.foodsubs.com/Ginger.html'), \
#                ('vegetable-cabbage', 'http://www.foodsubs.com/Cabbage.html'), \
#                ('vegetable-greens', 'http://www.foodsubs.com/Greensld.html'), \
#                ('vegetable-greens', 'http://www.foodsubs.com/Greenckg.html'), \
#                ('vegetable-flower', 'http://www.foodsubs.com/Vegiesinflor.html'), \
#                ('vegetable-bean', 'http://www.foodsubs.com/Snapbean.html'), \
#                ('vegetable-pods', 'http://www.foodsubs.com/Pods.html'), \
#                ('vegetable-peas', 'http://www.foodsubs.com/Shellpeas.html'), \
#                ('vegetable-ben', 'http://www.foodsubs.com/Shellbeans.html'), \
#                ('vegetable-mushrooms', 'http://www.foodsubs.com/Mushroom.html'), \
#                ('vegetable-fruits', 'http://www.foodsubs.com/Fruitvegies.html'), \
#                ('vegetable-tomatoes', 'http://www.foodsubs.com/Tomtom.html'), \
#                ('vegetable-eggplant', 'http://www.foodsubs.com/Eggplants.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squash.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squashsum.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squashasian.html'), \
#                ('vegetable-cucumber', 'http://www.foodsubs.com/Squcuke.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Peppersw.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Chilefre.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Peppersw.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Chiledry.html'), \
#                ('vegetable-olives', 'http://www.foodsubs.com/Olivpick.html'), \
#                ('vegetable-avocados', 'http://www.foodsubs.com/Avocados.html'), \
#                ('vegetable-sea', 'http://www.foodsubs.com/Seaveg.html'), \
#                ('vegetable-sprouts', 'http://www.foodsubs.com/Sprouts.html'), \
#                ('vegetable-other', 'http://www.foodsubs.com/Vegies.html'), \
#                ('fruit-citrus', 'http://www.foodsubs.com/Fruitcit.html'), \
#                ('fruit-berries', 'http://www.foodsubs.com/Fruitber.html'), \
#                ('fruit-stone', 'http://www.foodsubs.com/Fruitsto.html'), \
#                ('fruit-tropical', 'http://www.foodsubs.com/Fruittro.html'), \
#                ('fruit-tropical', 'http://www.foodsubs.com/Fruittroex.html'), \
#                ('fruit-melons', 'http://www.foodsubs.com/Fruitmel.html'), \
#                ('fruit-dry', 'http://www.foodsubs.com/Fruitdry.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Fruitoth.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Apples.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Pears.html'), \
#                ('fruit-preserves', 'http://www.foodsubs.com/Fruitpre.html'), \
#                ('fruit-candy', 'http://www.foodsubs.com/Candied.html'), \
#                ('fruit-juice', 'http://www.foodsubs.com/Juice.html'), \
#                ('dairy-milk', 'http://www.foodsubs.com/Dairyoth.html'), \
#                ('dairy-cultured', 'http://www.foodsubs.com/Cultmilk.html'), \
#                ('dairy-non', 'http://www.foodsubs.com/Nondairy.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheese.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chefresh.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chesoft.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chessoft.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chesfirm.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chefirm.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheblue.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheprocessed.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/CheeseAlt.html'), \
#                ('dairy-egg', 'http://www.foodsubs.com/Eggs.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsUniv.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAfrican.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAmerican.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAsian.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsEur.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsHisp.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsIndian.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsMiddleEast.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceUniv.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceAfr.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceAsian.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceEur.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceHisp.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceInd.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAfr.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAmer.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAsian.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixEur.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixHisp.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixInd.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixME.html'), \
#                ('seeds', 'http://www.foodsubs.com/Seeds.html'), \
#                ('seeds-grain', 'http://www.foodsubs.com/FGGrains.html'), \
#                ('seeds-rice', 'http://www.foodsubs.com/Rice.html'), \
#                ('seeds-wheat', 'http://www.foodsubs.com/GrainWheat.html'), \
#                ('seeds-corn', 'http://www.foodsubs.com/GrainCorn.html'), \
#                ('seeds-oats', 'http://www.foodsubs.com/GrainOats.html'), \
#                ('seeds-barley', 'http://www.foodsubs.com/GrainBarley.html'), \
#                ('seeds-buckwheat', 'http://www.foodsubs.com/GrainBuckwheat.html'), \
#                ('seeds-rye', 'http://www.foodsubs.com/GrainRye.html'), \
#                ('seeds', 'http://www.foodsubs.com/GrainTrit.html'), \
#                ('seeds-spelt', 'http://www.foodsubs.com/GrainSpelt.html'), \
#                ('seeds', 'http://www.foodsubs.com/Grainoth.html'), \
#                ('legumes', 'http://www.foodsubs.com/Peas.html'), \
#                ('legumes', 'http://www.foodsubs.com/Lentils.html'), \
#                ('legumes', 'http://www.foodsubs.com/Beans.html'), \
#                ('legumes', 'http://www.foodsubs.com/Snapbean.html'), \
#                ('legumes', 'http://www.foodsubs.com/Pods.html'), \
#                ('legumes-nuts', 'http://www.foodsubs.com/Nuts.html'), \
#                ('legumes', 'http://www.foodsubs.com/Nutseed.html'), \
#                ('legumes', 'http://www.foodsubs.com/Nutmeals.html'), \
#                ('legumes', 'http://www.foodsubs.com/Soyprod.html'), \
#                ('legumes', 'http://www.foodsubs.com/BeanProducts.html'), \
#                ('extracts', 'http://www.foodsubs.com/Extracts.html'), \
#                ('salt', 'http://www.foodsubs.com/Salt.html'), \
#                ('sugars', 'http://www.foodsubs.com/Sweeten.html'), \
#                ('sweetener', 'http://www.foodsubs.com/Syrups.html'), \
#                ('chocolate', 'http://www.foodsubs.com/Chocvan.html'), \
#                ('candy', 'http://www.foodsubs.com/Candy.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAf.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAmerican.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAsia.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntEur.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntHisp.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntInd.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntME.html'), \
#                ('vinegars', 'http://www.foodsubs.com/Vinegars.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurAnise.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurBitters.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurChoc.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurFruit.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurHerbal.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurMisc.html'), \
#                ('liquor', 'http://www.foodsubs.com/Liqueurs.html'), \
#                ('liquor', 'http://www.foodsubs.com/Aperitif.html'), \
#                ('liquor', 'http://www.foodsubs.com/Brandy.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRed.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesWhite.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesBlush.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesSparkling.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesDessert.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesFruit.html'), \
#                ('wine', 'http://www.foodsubs.com/Aperitif.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRice.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRice.html'), \
#                ('liquor', 'http://www.foodsubs.com/Liquor.html'), \
#                ('beer', 'http://www.foodsubs.com/Beer.html'), \
#                ('bitters', 'http://www.foodsubs.com/Bitters.html'), \
#                ('flours', 'http://www.foodsubs.com/Flour.html'), \
#                ('flours', 'http://www.foodsubs.com/Flournw.html'), \
#                ('flours', 'http://www.foodsubs.com/Nutmeals.html'), \
#                ('flours', 'http://www.foodsubs.com/Pasta.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaRods.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaRibbons.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaShapes.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaSoup.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaTubes.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaStuffed.html'), \
#                ('flours', 'http://www.foodsubs.com/Noodles.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesWheat.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesRice.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesAsianOther.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesOther.html'), \
#                ('flours', 'http://www.foodsubs.com/Dough.html'), \
#                ('thickeners', 'http://www.foodsubs.com/Thicken.html'), \
#                ('thickeners', 'http://www.foodsubs.com/ThickenStarch.html'), \
#                ('thickeners', 'http://www.foodsubs.com/ThickenGelatins.html'), \
#                ('miscellaneous', 'http://www.foodsubs.com/Wrappers.html'), \
#                ('bread', 'http://www.foodsubs.com/Bread.html'), \
#                ('cookies', 'http://www.foodsubs.com/Cookies.html'), \
#                ('cakes', 'http://www.foodsubs.com/Cakes.html'), \
#                ('crackers', 'http://www.foodsubs.com/Crackers.html'), \
#                ('bread', 'http://www.foodsubs.com/Crumbs.html'), \
#                ('bread', 'http://www.foodsubs.com/Flatbread.html'), \
#                ('meat-poultry', 'http://www.foodsubs.com/Poultry.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefChuck.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefRibs.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefLoin.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefRound.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefB&F.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefMisc.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkShoulder.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkLoin.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkLeg.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkSide.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkPicnic.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkMisc.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambShoulder.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambRibs.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambLoin.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambLeg.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambBreast.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambMisc.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealShoulder.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealRibs.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealLoin.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealLeg.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealBreast.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealMisc.html'), \
#                ('meat-other', 'http://www.foodsubs.com/MeatOther.html'), \
#                ('meat-other', 'http://www.foodsubs.com/Game.html'), \
#                ('meat-bacon', 'http://www.foodsubs.com/MeatcureBacon.html'), \
#                ('meat-sausage', 'http://www.foodsubs.com/MeatcureSausage.html'), \
#                ('meat-coldcuts', 'http://www.foodsubs.com/MeatcureCC.html'), \
#                ('meat-coldcuts', 'http://www.foodsubs.com/MeatcureHams.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarHeart.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarKidneys.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarLiver.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarMisc.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatDried.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Ffirmlea.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Ffirmfat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaCod.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaDor.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaDrum.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaFlat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaMull.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPerch.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPike.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPorg.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaRock.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaTile.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Fflakfat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Fishsmok.html'), \
#                ('seafood-shellfish', 'http://www.foodsubs.com/Shelfish.html'), \
#                ('seafood-shellfish', 'http://www.foodsubs.com/Shelfishcrab.html'), \
#                ('seafood-caviar', 'http://www.foodsubs.com/Caviar.html'), \
#                ('dairy-butters', 'http://www.foodsubs.com/Fatsoils.html'), \
#                ('oils', 'http://www.foodsubs.com/Oils.html'), \
#                ('flowers', 'http://www.foodsubs.com/Flowers.html'), \
#                ('miscellaneous', 'http://www.foodsubs.com/Misc.html')]

In [ ]:
#Scrape data and write to a csv (used above)

# fd = food_dictionary()
# for tup in scrape_list:
#     fd.turn_to_df(tup[0], tup[1])
# final_df = fd.df
# final_df.to_csv('final_recipe_list_categorized.csv', encoding='utf-8')